# Spring 2021

## Raquel Aoki

Starting the project


Source: https://github.com/JiajingZ/CopulaSensitivity

Comments: 
- 7.1 Section data is in R. 
- 7.2 Section is the GWAS study from Parkca, and the deconfounder. The data generated is different from Blei paper. 


In [3]:
#7.2 GWAS simulated study // sparse effects setting
#https://github.com/raquelaoki/ParKCa/blob/master/src/datapreprocessing.py

def sim_genes_TGP(Fs, ps, n_hapmapgenes, n_causes, n_units, S, D, randseed):
    '''
    #Adapted from Deconfounder's authors
    generate the simulated data
    input:
        - Fs, ps, n_hapmapgenes: not adopted in this example
        - n_causes = integer
        - n_units = m (columns)
        - S: PCA output n x 2
    '''
    np.random.seed(randseed)

    S = expit(S)
    Gammamat = np.zeros((n_causes, 3))
    Gammamat[:,0] = 0.2*npr.uniform(size=n_causes) #0.45
    Gammamat[:,1] = 0.2*npr.uniform(size=n_causes) #0.45
    Gammamat[:,2] = 0.05*np.ones(n_causes)
    S = np.column_stack((S[npr.choice(S.shape[0],size=n_units,replace=True),], \
        np.ones(n_units)))
    F = S.dot(Gammamat.T)
    #it was 2 instead of 1: goal is make SNPs binary
    G = npr.binomial(1, F)
    #unobserved group
    lambdas = KMeans(n_clusters=3, random_state=123).fit(S).labels_
    sG = sparse.csr_matrix(G)
    return G, lambdas


def generate_samples(SIMULATIONS,n_units,n_causes):
    '''
    Input:
    SIMULATIONS: number of datasets to be produced
    n_units, n_causes: dimentions
    Output (pickle format):
    snp_simulated datasets
    y: output simulated and truecases for each datset are together in a single matrix
    Note: There are options to load the data from vcf format and run the pca
    Due running time, we save the files and load from the pca.txt file
    '''
    #ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/supporting/hd_genotype_chip/
    #tgp_pca2.txt created in https://github.com/raquelaoki/ParKCa/blob/master/src/datapreprocessing.py
    S = np.loadtxt('data_s//tgp_pca2.txt', delimiter=',')

    sim_y = []
    sim_tc = []
    for sim in range(SIMULATIONS):
        G0, lambdas = sim_genes_TGP([], [], 0 , n_causes, n_units, S, 3, sim )
        G1, tc, y01 = sim_dataset(G0,lambdas, n_causes,n_units,sim)
        G = add_colnames(G1,tc)
        del G0,G1

        G.to_pickle('data_s//snp_simulated_'+str(sim)+'.txt')
        sim_y.append(y01)
        sim_tc.append(tc)
    sim_y = np.transpose(np.matrix(sim_y))
    sim_y = pd.DataFrame(sim_y)
    sim_y.columns = ['sim_'+str(sim) for sim in range(SIMULATIONS)]

    sim_tc = np.transpose(np.matrix(sim_tc))
    sim_tc = pd.DataFrame(sim_tc)
    sim_tc.columns = ['sim_'+str(sim) for sim in range(SIMULATIONS)]

    sim_y.to_pickle('data_s//snp_simulated_y01.txt')
    sim_tc.to_pickle('data_s//snp_simulated_truecauses.txt')


Todo: 
- Get old data
- Read the ammount of causes used by deconfudner


In [ ]:
#new simulated studies - binary nonlinear
#Reference: 
#https://github.com/JiajingZ/CopulaSensitivity/blob/CopSens/simulation/GaussianT_BinaryY_nonlinearYT/GaussianT_BinaryY_nonlinearYT_RR.R
#adapted from R to python
#GDP
#set seed
#variables initialization 
k = 4 #?
s = 1 #?
B = [2,0.5, -0.4, 0.2] #?
gamma = 2.8
sigma2_t, sigma2_y = 1, 1

tau_l = [3, -1, 1, -0.06] #linear effect
tau_nl = -4 #non linear effect
coef_true = tau_l.append(tau_nl)

def g_yt(t, tau_l, tau_nl): 
    '''
    t: t is n by k matrix
    '''
    #in R
    
    #col 3 
    t[3] = [item if item>0 else 0.7*item for item in t[3]] #t[,3] = ifelse(t[,3] > 0, t[,3], 0.7*t[,3])
    t = t.dot(tau_l) #t %*% tau_l + t[,1]^2 * tau_nl
    t[1] = (np.pow(t[1],2))*tau_nl
    
    return t

n = 80000
u = np.random.normal(mu = np.repeat(0,s), sigma = diag(s), n) 
tr = u.dot(np.transpose(B))
tr = tr + np.random.normal(mu = np.repeat(0,k), sigma = sigma2_t*diag(k), n) 
# y_tilde =  g_yt(t = tr) + u %*% gamma + rnorm(n, mean = 0, sd = sqrt(sigma2_y))????

y_tilde =  g_yt(t = tr) + u %*% gamma + rnorm(n, mean = 0, sd = sqrt(sigma2_y))
y = ifelse(y_tilde > 0, 1, 0)  ## binary Y
tr = data.frame(tr); colnames(tr) = c('t1', 't2', 't3', 't4')
y = as.numeric(y)

## theoretical values -------------------------------------------------------------
coef_mu_u_t <- t(B) %*% solve(B %*% t(B) + sigma2_t * diag(k))
sigma_u_t <- as.numeric(sqrt(1 - t(B) %*% solve(B %*% t(B) + sigma2_t * diag(k)) %*% B))
sigma_ytilde_t <- as.numeric(sqrt( gamma^2*sigma_u_t^2 + sigma2_y ))
sigma_ytilde_t_do <- as.numeric(sqrt( gamma^2 + sigma2_y ))

t_choice <- diag(k)
t2 = rep(0, k)

# true Treatment effect #
ytilde_mean_do <- g_yt(rbind(t_choice, t2))
y_mean_do <- c(pnorm(ytilde_mean_do/sigma_ytilde_t_do))
y_mean_do
effect_true <- y_mean_do[1:4]/y_mean_do[5]
effect_true

# true treatment effect bias #
ytilde_mean_do_bias <- c(rbind(t_choice, t2) %*% t(coef_mu_u_t) %*% gamma)

# true observed treatment effect #
ytilde_mean_obs <- ytilde_mean_do + ytilde_mean_do_bias
y_mean_obs <- c(pnorm(ytilde_mean_obs/sigma_ytilde_t))
y_mean_obs
effect_obs <- y_mean_obs[1:4]/y_mean_obs[5]
effect_obs
